In [1]:
import sqlite3
from sqlite3 import Error

import pandas as pd

from sqlalchemy import create_engine, Column, Integer, String

import datetime as dt

In [80]:
conn = sqlite3.connect('Data/isp.sqlite')

# Table creation

In [81]:
conn.execute('''CREATE TABLE Minneapolis_Centurylink
    (address_full varchar(100) PRIMARY KEY,
    major_city varchar(50),
    state varchar(4),
    lat float,
    lon float,
    block_group bigint,
    collection_datetime datetime,
    provider varchar(20),
    price double,
    speed_down float,
    speed_up float,
    speed_unit varchar(10),
    technology varchar(20),
    package varchar(100),
    fastest_speed_down float,
    fastest_speed_price double,
    speed_down_bins varchar(50),
    redlining_grade varchar(2),
    race_perc_non_white double,
    race_quantile varchar(50),
    median_household_income float,
    income_dollars_below_median float,
    income_level varchar(50),
    ppl_per_sq_mile double,
    n_providers int,
    internet_perc_broadband double
    );''')

OperationalError: table Minneapolis_Centurylink already exists

In [154]:
conn.execute('''CREATE TABLE Minneapolis_Centurylink_Grouped
    (
    block_group int,
    price double,
    speed_down float,
    speed_up float,
    speed_unit varchar(10),
    technology varchar(20),
    package varchar(100),
    fastest_speed_down float,
    fastest_speed_price double,
    speed_down_bins varchar(50),
    redlining_grade varchar(2),
    median_household_income float,
    income_dollars_below_median float,
    income_level varchar(50),
    ppl_per_sq_mile double,
    n_providers int,
    internet_perc_broadband double,
    latstring varchar(5000),
    lonstring varchar(5000),
    primary key(block_group, speed_down, speed_up, redlining_grade, speed_down_bins, fastest_speed_down, fastest_speed_price, package)
    );''')

In [153]:
# For dropping tables as needed

conn.execute('''DROP TABLE Minneapolis_Centurylink_Grouped;''')

# Full table creation

In [4]:
minneapolis_df = pd.read_csv('Data/minneapolis_centurylink_plans.csv')

In [5]:
minneapolis_df.head()

,address_full,major_city,state,lat,lon,block_group,collection_datetime,provider,price,speed_down,...,speed_down_bins,redlining_grade,race_perc_non_white,race_quantile,median_household_income,income_dollars_below_median,income_level,ppl_per_sq_mile,n_providers,internet_perc_broadband
0,"3739 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934880,-93.225727,270531088001,1650164907,CenturyLink,50.0,200.0,...,Blazing (≥200),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3.0,0.608025
1,"3636 34TH AVE S,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.936622,-93.223331,270531088001,1650164906,CenturyLink,50.0,200.0,...,Blazing (≥200),B,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3.0,0.608025
2,"3740 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934692,-93.226591,270531088001,1650164915,CenturyLink,50.0,200.0,...,Blazing (≥200),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3.0,0.608025
3,"3759 1/2 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934468,-93.225210,270531088001,1650164918,CenturyLink,50.0,30.0,...,Medium (25-99),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3.0,0.608025
4,"3741 MINNEHAHA AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934691,-93.223820,270531088001,1650164916,CenturyLink,50.0,200.0,...,Blazing (≥200),C,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3.0,0.608025


In [6]:
minneapolis_df['collection_datetime'] = pd.to_datetime(minneapolis_df['collection_datetime'], unit='s', utc=True).map(lambda x: x.tz_convert('America/Chicago'))

In [7]:
minneapolis_df.count()

address_full                   11583
major_city                     11583
state                          11583
lat                            11583
lon                            11583
block_group                    11583
collection_datetime            11583
provider                       11583
price                          11583
speed_down                     11583
speed_up                       11583
speed_unit                     11583
technology                     11583
package                        11583
fastest_speed_down             11583
fastest_speed_price            11583
speed_down_bins                11583
redlining_grade                 9279
race_perc_non_white            11583
race_quantile                  11583
median_household_income        11093
income_dollars_below_median    11093
income_level                   11093
ppl_per_sq_mile                11583
n_providers                    11583
internet_perc_broadband        11583
dtype: int64

In [8]:
minneapolis_df['redlining_grade'] = minneapolis_df['redlining_grade'].fillna('None')

minneapolis_df['median_household_income'] = minneapolis_df['median_household_income'].fillna('Unknown')
minneapolis_df['income_level'] = minneapolis_df['income_level'].fillna('Unknown')

In [12]:
minneapolis_df.count()

address_full                   11583
major_city                     11583
state                          11583
lat                            11583
lon                            11583
block_group                    11583
collection_datetime            11583
provider                       11583
price                          11583
speed_down                     11583
speed_up                       11583
speed_unit                     11583
technology                     11583
package                        11583
fastest_speed_down             11583
fastest_speed_price            11583
speed_down_bins                11583
redlining_grade                11583
race_perc_non_white            11583
race_quantile                  11583
median_household_income        11583
income_dollars_below_median    11093
income_level                   11583
ppl_per_sq_mile                11583
n_providers                    11583
internet_perc_broadband        11583
dtype: int64

In [10]:
minneapolis_df.to_sql('Minneapolis_Centurylink', conn, if_exists='append', index= False)

11583

# Grouped table creation 

In [162]:
groupedCoords_df = minneapolis_df.groupby(['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])['lat', 'lon'].agg(lats=('lat', 'unique'),
                                                                    lons=('lon', 'unique'))

# groupedCoords_df["'lat', 'unique'"].rename('lats')
groupedCoords_df.head()


C:\Users\Dan\AppData\Local\Temp\ipykernel_28624\3839830684.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  groupedCoords_df = minneapolis_df.groupby(['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])['lat', 'lon'].agg(lats=('lat', 'unique'),


lats  \
block_group  redlining_grade speed_down speed_up package                                                                                 
270530001011 D               10.0       0.75     HSI Upto 10 Mbps/750 Kbps Prepaid                             [45.049852, 45.0500186]   
                             40.0       3.00     HSI Upto 40 Mbps/3 Mbps Prepaid                                          [45.0496982]   
                             80.0       10.00    HSI Upto 80 Mbps/10 Mbps Prepaid    [45.0499144, 45.0498047, 45.0480286, 45.048006...   
                             140.0      20.00    HSI Upto 140 Mbps/20 Mbps Prepaid                                        [45.0502983]   
                             200.0      200.00   HSI Upto 200 Mbps/200 Mbps Prepaid  [45.0506428, 45.0495296, 45.0496982, 45.049206...   

                                                                                                                                  lons  
block_group  redlining_grade speed_down speed_up package                                                                                
270530001011 D               10.0       0.75     HSI Upto 10 Mbps/750 Kbps Prepaid                                       [-93.3033502]  
                             40.0       3.00     HSI Upto 40 Mbps/3 Mbps Prepaid                                         [-93.3013453]  
                             80.0       10.00    HSI Upto 80 Mbps/10 Mbps Prepaid    [-93.3000893, -93.3000891, -93.2995718, -93.30...  
                             140.0      20.00    HSI Upto 140 Mbps/20 Mbps Prepaid                                        [-93.307462]  
                             200.0      200.00   HSI Upto 200 Mbps/200 Mbps Prepaid  [-93.3020727, -93.3038677, -93.304622, -93.305...

In [150]:
groupedCoords_df.size

3102

In [186]:
lats = minneapolis_df.groupby(['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])\
['lat'].apply(list).reset_index()

lons = minneapolis_df.groupby(['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])\
['lon'].apply(list).reset_index()

In [188]:
groupedCoords_df = lats.merge(lons, on=['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])

In [159]:
unique_df = minneapolis_df[['block_group', 'price', 'speed_down', 'speed_up', 'speed_unit', 'technology', 'package', \
                    'fastest_speed_down', 'fastest_speed_price', 'speed_down_bins', 'redlining_grade', \
                    'median_household_income', 'income_level', 'n_providers', 'internet_perc_broadband']].drop_duplicates()

unique_df.count()

block_group                1553
price                      1553
speed_down                 1553
speed_up                   1553
speed_unit                 1553
technology                 1553
package                    1553
fastest_speed_down         1553
fastest_speed_price        1553
speed_down_bins            1553
redlining_grade            1553
median_household_income    1553
income_level               1553
n_providers                1553
internet_perc_broadband    1553
dtype: int64

In [189]:
merged_df = unique_df.merge(groupedCoords_df, on=['block_group', 'redlining_grade', 'speed_down', 'speed_up', 'package'])


merged_df.head()

,block_group,price,speed_down,speed_up,speed_unit,technology,package,fastest_speed_down,fastest_speed_price,speed_down_bins,redlining_grade,median_household_income,income_level,n_providers,internet_perc_broadband,lat,lon
0,270531088001,50.0,200.0,200.0,Mbps,Fiber,HSI Upto 200 Mbps/200 Mbps Prepaid,940.0,65.0,Blazing (≥200),D,60417.0,Middle-Lower,3.0,0.608025,"[44.9348799, 44.9346924, 44.9364954, 44.937001...","[-93.2257266, -93.2265906, -93.2269033, -93.22..."
1,270531088001,50.0,200.0,200.0,Mbps,Fiber,HSI Upto 200 Mbps/200 Mbps Prepaid,940.0,65.0,Blazing (≥200),B,60417.0,Middle-Lower,3.0,0.608025,"[44.9366218, 44.9365129, 44.936838]","[-93.2233313, -93.2233306, -93.2239298]"
2,270531088001,50.0,30.0,1.5,Mbps,Not Fiber,HSI Upto 30 Mbps/1.5 Mbps Prepaid,30.0,50.0,Medium (25-99),D,60417.0,Middle-Lower,3.0,0.608025,[44.9344676],[-93.2252103]
3,270531088001,50.0,200.0,200.0,Mbps,Fiber,HSI Upto 200 Mbps/200 Mbps Prepaid,940.0,65.0,Blazing (≥200),C,60417.0,Middle-Lower,3.0,0.608025,"[44.9346907, 44.9365366, 44.9351768, 44.9345467]","[-93.2238197, -93.2246345, -93.2241686, -93.22..."
4,270530095002,50.0,200.0,200.0,Mbps,Fiber,HSI Upto 200 Mbps/200 Mbps Prepaid,940.0,65.0,Blazing (≥200),B,50278.0,Middle-Lower,3.0,0.869919,"[44.9348538, 44.9365321, 44.9346378, 44.936396...","[-93.2646029, -93.2646121, -93.2628871, -93.26..."


In [191]:
# df['liststring'] = [','.join(map(str, l)) for l in df['lists']]


merged_df['latstring'] = [','.join(map(str, l)) for l in merged_df['lat']]
merged_df['lonstring'] = [','.join(map(str, l)) for l in merged_df['lon']]


In [192]:
merged_string_df = merged_df[['block_group', 'price', 'speed_down', 'speed_up', 'speed_unit', 'technology', 'package', \
                    'fastest_speed_down', 'fastest_speed_price', 'speed_down_bins', 'redlining_grade', \
                    'median_household_income', 'income_level', 'n_providers', 'internet_perc_broadband', 'latstring', 'lonstring']].drop_duplicates()

In [194]:
merged_string_df.to_sql('Minneapolis_Centurylink_Grouped', conn, if_exists='append', index= False)

1553

# Test queries

In [143]:
from pprint import pprint 

In [145]:
c = conn.cursor()

results = c.execute('''SELECT lat, lon from Minneapolis_CenturyLink where block_group = 270531114003''').fetchall()


pprint(results)

[(44.9054552, -93.2950123),
 (44.9075343, -93.2923365),
 (44.9053732, -93.2904429),
 (44.9060193, -93.2937352),
 (44.9057941, -93.2917302),
 (44.9066169, -93.2911828),
 (44.907446, -93.2895469),
 (44.9062074, -93.2924562),
 (44.905303, -93.2887235),
 (44.9055156, -93.2917299),
 (44.9056797, -93.2950124),
 (44.90705, -93.2968317),
 (44.9052384, -93.2892214),
 (44.9071892, -93.2968332),
 (44.9059295, -93.2911823),
 (44.9057911, -93.2950124),
 (44.9083885, -93.2969121),
 (44.9076673, -93.2939731),
 (44.90525, -93.2930028),
 (44.9052484, -93.292455),
 (44.9075699, -93.2957557),
 (44.9075232, -93.2921024),
 (44.9057936, -93.2911821),
 (44.9074501, -93.2897728),
 (44.9062174, -93.2962834),
 (44.9075714, -93.2949231),
 (44.9074786, -93.2904151),
 (44.9075508, -93.2951146)]


In [7]:
names = list(map(lambda x: x[0], c.description))

data = pd.DataFrame(results, columns=names)

In [8]:
data.head()

,address_full,major_city,state,lat,lon,block_group,collection_datetime,provider,price,speed_down,...,speed_down_bins,redlining_grade,race_perc_non_white,race_quantile,median_household_income,income_dollars_below_median,income_level,ppl_per_sq_mile,n_providers,internet_perc_broadband
0,"3739 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934880,-93.225727,270531088001,1650164907,CenturyLink,50.0,200.0,...,Blazing (≥200),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3,0.608025
1,"3636 34TH AVE S,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.936622,-93.223331,270531088001,1650164906,CenturyLink,50.0,200.0,...,Blazing (≥200),B,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3,0.608025
2,"3740 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934692,-93.226591,270531088001,1650164915,CenturyLink,50.0,200.0,...,Blazing (≥200),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3,0.608025
3,"3759 1/2 SNELLING AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934468,-93.225210,270531088001,1650164918,CenturyLink,50.0,30.0,...,Medium (25-99),D,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3,0.608025
4,"3741 MINNEHAHA AVE,MINNEAPOLIS,MN 55406,USA",minneapolis,MN,44.934691,-93.223820,270531088001,1650164916,CenturyLink,50.0,200.0,...,Blazing (≥200),C,0.640575,least white,60417.0,5651.0,Middle-Lower,3329.25597,3,0.608025


In [148]:
c = conn.cursor()

results = c.execute('''SELECT latstring, lonstring from Minneapolis_CenturyLink_Grouped where block_group = 270531114003''').fetchall()

results

[('44.9054552,44.9075343,44.9053732,44.9060193,44.9057941,44.9066169,44.907446,44.9062074,44.905303,44.9055156,44.9056797,44.90705,44.9052384,44.9071892,44.9059295,44.9057911,44.9083885,44.9076673,44.90525,44.9052484,44.9075699,44.9075232,44.9057936,44.9074501,44.9062174,44.9075714,44.9074786,44.9075508',
  '-93.2950123,-93.2923365,-93.2904429,-93.2937352,-93.2917302,-93.2911828,-93.2895469,-93.2924562,-93.2887235,-93.2917299,-93.2950124,-93.2968317,-93.2892214,-93.2968332,-93.2911823,-93.2969121,-93.2939731,-93.2930028,-93.292455,-93.2957557,-93.2921024,-93.2911821,-93.2897728,-93.2962834,-93.2949231,-93.2904151,-93.2951146')]

In [78]:
conn.close()

In [10]:
names

['address_full',
 'major_city',
 'state',
 'lat',
 'lon',
 'block_group',
 'collection_datetime',
 'provider',
 'price',
 'speed_down',
 'speed_up',
 'speed_unit',
 'technology',
 'package',
 'fastest_speed_down',
 'fastest_speed_price',
 'speed_down_bins',
 'redlining_grade',
 'race_perc_non_white',
 'race_quantile',
 'median_household_income',
 'income_dollars_below_median',
 'income_level',
 'ppl_per_sq_mile',
 'n_providers',
 'internet_perc_broadband']